In [173]:
import random, math
import numpy as np

def sigmoid(x):
    return 1/(1+np.exp(-x))

def swish(x):
    return x*sigmoid(x)

def relu(x):
    return np.maximum(0,x)

def softmax(x):
    p = np.exp(x - np.max(x))
    return p/np.sum(p)

def activation_function(z,act):
    if act=="sigmoid":
        return sigmoid(z)
    elif act == "swish":
        return swish(z)
    elif act == "relu":
        return relu(z)
    elif act=="tanh":
        return np.tanh(z)
    elif act=="softmax":
        return softmax(z)
    
    
    
    

class RNN():
    
    def __init__(self, l_rate):
        self.layers = []
        self.learning_rate = l_rate
        
    def add_layer(self, input_size, output_size, activation=None, last_activation=None):
        new_layer = Layer(input_size, output_size, activation)
        self.layers.append(new_layer)
        
    def forward_pass(self, input_data):
        self.layers[0].forward_pass(input_data)
        for i in range(1, len(self.layers)):
            prev_layer_out = self.layers[i-1].Y[-1]
            self.layers[i].forward_pass(prev_layer_out)
            
        return self.layers[-1].Y[-1]
            
    def clear_memory(self):
        for i in range(0, len(self.layers)):
            self.layers[i].clear_memory()
            
            
            
class Layer():
    
    def __init__(self, input_size, output_size, activation=None, last_activation=None):
        self.input_size = input_size
        self.output_size = output_size
        self.activation = activation
        self.last_activation = last_activation
        
        self.weights_X = self.weights_init(output_size, input_size)
        self.weights_H = self.weights_init(output_size, output_size)
        self.weights_Y = self.weights_init(output_size, output_size)
        self.bias      = self.weights_init(output_size, 1)
        
        self.A = np.empty((0,output_size,1))
        self.Y = np.empty((0,output_size,1))
        self.H = np.empty((0,output_size,1))
        
        
    def weights_init(self, rows, cols):
        return np.random.uniform(-1,1,(rows, cols))
    
    
    def forward_pass(self, input_data):
        new_H = self.weights_X.dot(input_data) + self.bias
        if len(self.A) > 0:
             new_H += self.weights_H.dot(self.A[-1]) 
        self.H = np.append(self.H, np.array([new_H]), axis=0)
        
        
        new_A = activation_function(self.H[-1], self.activation)
        self.A = np.append(self.A, np.array([new_A]), axis=0)
        
        
        new_Y = self.weights_Y.dot(self.A[-1])
        if self.last_activation != None:
            new_Y = activation_function(new_Y, self.last_activation)
        self.Y = np.append(self.Y, np.array([new_Y]), axis=0)
        

    def clear_memory(self):
        self.A = np.empty((0,self.output_size,1))
        self.Y = np.empty((0,self.output_size,1))
        self.H = np.empty((0,self.output_size,1))
        
        
    

In [174]:
network = RNN(0.01)
network.add_layer(1, 3, "tanh")
network.add_layer(3, 2, "tanh")
network.add_layer(2, 1, "tanh")

In [175]:
input_data = [
    [[0.0]],
    [[0.1]],
    [[0.2]],
    [[0.3]],
    [[0.4]],
    [[0.5]],
    [[0.6]],
    [[0.7]]
]

In [176]:
network.clear_memory()
for i in range(0, len(input_data)-1):
    network.forward_pass(input_data[i])

(1, 3, 1)
(1, 2, 1)
(1, 1, 1)
(2, 3, 1)
(2, 2, 1)
(2, 1, 1)
(3, 3, 1)
(3, 2, 1)
(3, 1, 1)
(4, 3, 1)
(4, 2, 1)
(4, 1, 1)
(5, 3, 1)
(5, 2, 1)
(5, 1, 1)
(6, 3, 1)
(6, 2, 1)
(6, 1, 1)
(7, 3, 1)
(7, 2, 1)
(7, 1, 1)
